In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c fastchan fastai

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [3]:
import json

In [4]:
with open("bidder-data/data152.json") as f:
    ex = json.load(f)

In [5]:
from game_engine import *

In [6]:
def get_data_point(record):
    board = get_board_from_identifier(record["board"])
    dealer = board["dealer"]
    return list(map(CARDS.index, board["hands"][dealer])), CALLS.index(record["calls"][0])

In [19]:
from fastai.data.all import *

In [7]:
get_data_point(ex[0])

([1, 7, 9, 16, 18, 20, 28, 31, 41, 43, 45, 47, 50], 0)

In [8]:
ds = Datasets(ex, 

In [9]:
dl = DataLoader(ds, shuffle=True)

In [10]:
class FirstCall(nn.Module):
    def __init__(self):
        super(FirstCall, self).__init__()
        embed_dims = 16
        hidden_count = 32
        self.stack = nn.Sequential(
            nn.EmbeddingBag(len(CARDS), embed_dims, mode="sum"),
            nn.Linear(embed_dims, hidden_count),
            nn.ReLU(),
            nn.Linear(hidden_count, len(CALLS)),
        )

    def forward(self, x):
        logits = self.stack(x)
        return logits

In [11]:
model = FirstCall()

In [12]:
print(model)

FirstCall(
  (stack): Sequential(
    (0): EmbeddingBag(52, 16, mode=sum)
    (1): Linear(in_features=16, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=38, bias=True)
  )
)


In [13]:
xs = torch.tensor([x for x, y in ds])

In [14]:
logits = model(xs)

In [15]:
logits.shape

torch.Size([500, 38])

In [16]:
len(CALLS)

38

In [24]:
def get_x(record):
    board = get_board_from_identifier(record["board"])
    dealer = board["dealer"]
    return board["hands"][dealer]

In [23]:
def get_y(record):
    return record["calls"][0]

In [36]:
calls = DataBlock(
    (MultiCategoryBlock(vocab=CARDS), CategoryBlock(vocab=CALLS)),
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_x=get_x,
    get_y=get_y)

In [37]:
dls = calls.dataloaders(ex)

In [38]:
dls.valid.show_batch(max_n=4, nrows=1)

2C;4C;AC;2D;5D;8D;QD;4H;JH;QH;2S;4S;QS
4C;KC;2D;5D;7D;9D;3H;4H;7H;AH;2S;QS;AS
5C;6C;AC;3D;6D;8D;2H;7H;QH;5S;6S;TS;JS
2C;6C;7C;9C;KC;2D;6D;JD;4H;6H;9H;3S;5S
P
1D
P
P
